In [1]:
%gui qt
%load_ext autoreload
%autoreload 2
    
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.Bnd import Bnd_OBB, Bnd_Box
from OCC.Core.BRepBuilderAPI import (
    BRepBuilderAPI_Transform,
    BRepBuilderAPI_MakeEdge,
    BRepBuilderAPI_MakeWire,
    BRepBuilderAPI_MakeFace,
)
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopoDS import TopoDS_Compound, TopoDS_Wire, TopoDS_Vertex, TopoDS_Shape, TopoDS_Face
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.gp import gp_Trsf, gp_Pnt, gp_Ax3, gp_Ax1, gp_Dir, gp_Pln, gp_Vec, gp_Mat, gp_Quaternion
from OCC.Display.SimpleGui import init_display
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from IPython.display import display, HTML
from math import acos, degrees
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.Geom import Geom_CylindricalSurface


import pandas as pd
import numpy as np
import json
import math

# Start display without blocking
viewer, start_display, add_menu, add_function_to_menu = init_display()

# JSON Library path
json_path = "../data/Shape_classifier_info.json"

viewer.EraseAll()
viewer.View_Iso()
viewer.FitAll()

pyside6 backend - Qt version 6.8.3


In [2]:
# Load the STEP and extract 'solid'
def load_step(file_path: str):
    reader = STEPControl_Reader()
    status = reader.ReadFile(file_path)
    if status != 1:
        raise RuntimeError(f"Failed to read STEP file: {file_path}")
    reader.TransferRoots()
    return reader.Shape()

In [3]:
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from OCC.Core.AIS import AIS_Shape

def visualize(shape, display, color="CYAN", alpha=None, clear=False):
    """
    Displays a shape with global XYZ axes and OBB extents.
    
    Args:
        shape: The TopoDS_Shape to display.
        display: OCC viewer handle.
        color: Optional string color name (default "CYAN").
        alpha: Optional float (0.0 - 1.0) for transparency (0 = fully transparent, 1 = opaque).
        clear: If True, erase all previous shapes before displaying.
    """

    def get_occ_color(color_name):
        colors = {
            "RED": (1.0, 0.0, 0.0),
            "GREEN": (0.0, 1.0, 0.0),
            "BLUE": (0.0, 0.0, 1.0),
            "PSS_DARK_BLUE": (0.023, 0.105, 0.215),
            "PSS_LIGHT_BLUE": (0.592, 0.792, 0.921),
            "YELLOW": (1.0, 1.0, 0.0),
            "ORANGE": (1.0, 0.5, 0.0),
            "CYAN": (0.0, 1.0, 1.0),
            "MAGENTA": (1.0, 0.0, 1.0),
            "WHITE": (1.0, 1.0, 1.0),
            "GRAY": (0.5, 0.5, 0.5),
            "BLACK": (0.0, 0.0, 0.0),
        }
        r, g, b = colors.get(color_name.upper(), (1.0, 1.0, 1.0))
        return Quantity_Color(r, g, b, Quantity_TOC_RGB)

    if clear:
        display.Context.EraseAll()

    # Convert color name to OCC color
    qcolor = get_occ_color(color)

    # Wrap in AIS_Shape to support transparency
    ais_shape = AIS_Shape(shape)
    ais_shape.SetColor(qcolor)

    if alpha is not None:
        ais_shape.SetTransparency(1.0 - alpha)  # OCC expects "opacity", not alpha
        ais_shape.SetDisplayMode(1)  # Shaded

    display.Context.Display(ais_shape, True)
    display.FitAll()


In [4]:
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def draw_obb_box(obb, viewer, color="ORANGE", update=True):
    """
    Draws the 12 edges of the OBB as lines in the viewer.
    """
    c = obb['center']
    cx, cy, cz = c.X(), c.Y(), c.Z()
    hx, hy, hz = obb['half_lengths'][0], obb['half_lengths'][1], obb['half_lengths'][2]
    dx, dy, dz = obb['dir_x'], obb['dir_y'], obb['dir_z']

    # Precompute corner offsets
    offsets = [
        (+hx, +hy, +hz), (-hx, +hy, +hz), (-hx, -hy, +hz), (+hx, -hy, +hz),
        (+hx, +hy, -hz), (-hx, +hy, -hz), (-hx, -hy, -hz), (+hx, -hy, -hz)
    ]

    # Compute corners
    corners = []
    for ox, oy, oz in offsets:
        x = cx + ox * dx.X() + oy * dy.X() + oz * dz.X()
        y = cy + ox * dx.Y() + oy * dy.Y() + oz * dz.Y()
        z = cz + ox * dx.Z() + oy * dy.Z() + oz * dz.Z()
        corners.append(gp_Pnt(x, y, z))

    # Define edges by corner indices
    edges_idx = [
        (0, 1), (1, 2), (2, 3), (3, 0),  # top face
        (4, 5), (5, 6), (6, 7), (7, 4),  # bottom face
        (0, 4), (1, 5), (2, 6), (3, 7),  # vertical edges
    ]

    for i1, i2 in edges_idx:
        edge = BRepBuilderAPI_MakeEdge(corners[i1], corners[i2]).Edge()
        viewer.DisplayShape(edge, color=color, update=False)

    if update:
        viewer.FitAll()

In [5]:
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE, TopAbs_EDGE
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve, BRepAdaptor_Surface
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Vec, gp_Ax3, gp_Trsf
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def robust_align_solid_from_geometry(solid, tol=1e-3):
    """
    Aligns a solid such that:
    - X axis is the longest edge on the largest planar face
    - Y axis is the normal of that face
    - Z axis is X × Y (right-handed)
    Returns:
    - aligned solid
    - transformation used
    - local coordinate system (gp_Ax3)
    - largest face used
    - dir_x, dir_y, dir_z
    """
    # Step 1: Find the largest planar face
    explorer = TopExp_Explorer(solid, TopAbs_FACE)
    largest_face = None
    max_area = 0.0

    while explorer.More():
        face = explorer.Current()
        props = GProp_GProps()
        brepgprop.SurfaceProperties(face, props)
        area = props.Mass()
        if area > max_area:
            max_area = area
            largest_face = face
        explorer.Next()

    if not largest_face:
        raise RuntimeError("No planar face found in solid.")

    # Step 2: Get longest edge on that face → X axis
    edge_exp = TopExp_Explorer(largest_face, TopAbs_EDGE)
    longest_vec = None
    max_length = 0.0

    while edge_exp.More():
        edge = edge_exp.Current()
        curve = BRepAdaptor_Curve(edge)
        p1 = curve.Value(curve.FirstParameter())
        p2 = curve.Value(curve.LastParameter())
        vec = gp_Vec(p1, p2)
        length = vec.Magnitude()
        if length > max_length:
            max_length = length
            longest_vec = vec
        edge_exp.Next()

    if not longest_vec or longest_vec.Magnitude() < tol:
        raise RuntimeError("Failed to find valid longest edge.")

    dir_x = gp_Dir(longest_vec)

    # Step 3: Get face normal → Y axis
    surf_adapt = BRepAdaptor_Surface(largest_face)
    if surf_adapt.GetType() != 0:  # not a plane
        raise RuntimeError("Largest face is not planar.")
    dir_y = surf_adapt.Plane().Axis().Direction()

    # Step 4: Compute Z = X × Y, and re-orthogonalize Y to X and Z
    dir_z = dir_x.Crossed(dir_y)
    dir_y = dir_z.Crossed(dir_x)  # ensure orthogonality

    # Step 5: Create transformation from this local frame to global (X,Y,Z)
    origin = gp_Pnt(0, 0, 0)
    from_cs = gp_Ax3(origin, dir_x, dir_y)
    to_cs = gp_Ax3(origin, gp_Dir(1, 0, 0), gp_Dir(0, 1, 0))  # global X/Y/Z

    trsf = gp_Trsf()
    trsf.SetDisplacement(from_cs, to_cs)

    aligned = BRepBuilderAPI_Transform(solid, trsf, True).Shape()
    
    return aligned, trsf, to_cs, largest_face, dir_x, dir_y, dir_z


In [6]:
from OCC.Core.gp import gp_Ax3, gp_Trsf, gp_Pnt, gp_Dir
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def align_to_dstv_frame(solid, origin_local, dir_x, dir_y, dir_z):
    """
    Transforms a solid so that:
    - dir_x aligns to global X (beam length)
    - dir_y aligns to global Y (beam height)
    - dir_z aligns to global Z (beam width)
    - origin_local maps to global (0, 0, 0)
    """
    # Create local coordinate system from solid geometry
    local_ax3 = gp_Ax3(origin_local, gp_Dir(*dir_x), gp_Dir(*dir_y))

    # Create global DSTV coordinate system
    global_ax3 = gp_Ax3(gp_Pnt(0, 0, 0), gp_Dir(1, 0, 0), gp_Dir(0, 1, 0))

    # Create transformation
    trsf = gp_Trsf()
    trsf.SetDisplacement(local_ax3, global_ax3)

    # Apply to shape
    transformer = BRepBuilderAPI_Transform(solid, trsf, True)
    return transformer.Shape(), trsf


In [7]:
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir

def compute_dstv_origin(center: gp_Pnt, extents: list, dir_x: gp_Dir, dir_y: gp_Dir, dir_z: gp_Dir):
    """
    Computes the DSTV origin (rear-bottom-left) for a shape aligned via OBB.

    Parameters:
    - center: gp_Pnt — the center of the OBB (post-alignment)
    - extents: [length, height, width] — full extents along dir_x, dir_y, dir_z
    - dir_x: gp_Dir — length/feed direction
    - dir_y: gp_Dir — height/web direction
    - dir_z: gp_Dir — width/flange direction

    Returns:
    - gp_Pnt — rear-bottom-left corner in the aligned local frame
    """

    # Compute offset from center to rear-bottom-left corner
    dx = -extents[0] / 2  # back along length
    dy = -extents[1] / 2  # down along height
    dz = -extents[2] / 2  # left along width

    # Offset vector from center to origin
    offset_vec = gp_Vec(dir_x).Scaled(dx) + gp_Vec(dir_y).Scaled(dy) + gp_Vec(dir_z).Scaled(dz)

    # Translate center to origin
    origin_local = center.Translated(offset_vec)
    return origin_local


In [8]:
# import json

def classify_profile(cs, json_path, tol_dim=1.0, tol_area=0.05):
    """
    Attempts to classify a structural profile by matching its dimensions and area.
    Tries both the original and swapped width/height to detect if a 90° rotation is needed.
    """
    with open(json_path) as f:
        lib = json.load(f)

    def try_match(height, width):
        best, best_score = None, float('inf')
        for cat, ents in lib.items():
            for name, info in ents.items():
                dh = abs(height - info["height"])
                dw = abs(width  - info["width"])
                if dh > tol_dim or dw > tol_dim:
                    continue
                ea = abs(cs["area"] - info["csa"]) / info["csa"]
                if ea > tol_area:
                    continue
                el = abs(cs["length"] - info.get("length", cs["length"])) / info.get("length", cs["length"])
                score = dh + dw + ea * 100 + el * 100
                if score < best_score:
                    best_score = score
                    best = {
                        "Designation": name,
                        "Category": cat,
                        "Profile_type": info["code_profile"],
                        "Match_score": score,
                        "Requires_rotation": False,  # to be set later
                    
                        "JSON": {
                            "height": info["height"],
                            "width": info["width"],
                            "csa": info["csa"],
                            "length": 0,
                            "Mass": info.get("mass", 0.0)*cs["length"]/1000,
                            "web_thickness": info.get("web_thickness", 0.0),
                            "flange_thickness": info.get("flange_thickness", 0.0),
                            "root_radius": info.get("root_radius", 0.0),
                            "toe_radius":  info.get("toe_radius", 0.0)
                        },
                        "STEP": {
                            "height": height,
                            "width": width,
                            "area": cs["area"],
                            "length": info.get("length", cs["length"]),
                            "mass": info.get("mass", 0.0)*cs["length"]/1000,
                            "web_thickness": info.get("web_thickness", 0.0),
                            "flange_thickness": info.get("flange_thickness", 0.0),
                            "root_radius": info.get("root_radius", 0.0),
                            "toe_radius":  info.get("toe_radius", 0.0)
                        }
                    }
        return best, best_score

    # Try both normal and swapped width/height
    original, score1 = try_match(cs["span_web"], cs["span_flange"])
    swapped, score2  = try_match(cs["span_flange"], cs["span_web"])

    header_dict = {


    }
   
    if original and (not swapped or score1 <= score2):
        original["Requires_rotation"] = False
        return original
    elif swapped:
        swapped["Requires_rotation"] = True
        return swapped
    else:
        return None


In [9]:
def print_result_table(result, precision=2):
    """
    Pretty print classification result as a one-row DataFrame with rounded floats.
    """
    if not result:
        print("No classification result.")
        return

    step_vals = result["STEP"]
    json_vals = result["JSON"]
    
    data = {
        "Field": [
            "Designation", "Category", "Profile Type", "Match Score", "Rotation Required",
            "Height (mm)", "Width (mm)", "CSA (mm²)", "Length (mm)", "Mass (Kg)",
            "Web Thickness (mm)", "Flange Thickness (mm)", "Root Radius (mm)", "Toe Radius (mm)"
        ],
        "STEP File": [
            result["Designation"],
            result["Category"],
            result["Profile_type"],
            f"{result['Match_score']:.2f}",
            result["Requires_rotation"],
            f"{step_vals['height']:.2f}",
            f"{step_vals['width']:.2f}",
            f"{step_vals['area']:.2f}",
            f"{step_vals['length']:.2f}",
            f"{step_vals['mass']:.2f}",
            "","","",""
        ],
        "JSON Spec": [
            "", "", "", "", "",
            f"{json_vals['height']:.2f}",
            f"{json_vals['width']:.2f}",
            f"{json_vals['csa']:.2f}",
            f"{json_vals['length']:.2f}",
            "",
            f"{json_vals['web_thickness']:.2f}",
            f"{json_vals['flange_thickness']:.2f}",
            f"{json_vals['root_radius']:.2f}",
            f"{json_vals['toe_radius']:.2f}"
        ]
    }

    df = pd.DataFrame(data)
    display(df.style.set_properties(**{
        'text-align': 'left'
    }).set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'left')]
    }]))



In [10]:
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.gp import gp_Pnt, gp_Dir
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop

def compute_section_area(shape):
    props = GProp_GProps()
    brepgprop.SurfaceProperties(shape, props)
    return props.Mass()

def compute_axis_aligned_bounding_box(shape):
    """
    Computes a world-aligned bounding box (AABB) and includes true section area.
    """
    bbox = Bnd_Box()
    brepbndlib.Add(shape, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()

    # Half-sizes
    hx = (xmax - xmin) / 2
    hy = (ymax - ymin) / 2
    hz = (zmax - zmin) / 2

    center = gp_Pnt((xmin + xmax)/2, (ymin + ymax)/2, (zmin + zmax)/2)
    
    # Define frame as default world axes
    dir_x = gp_Dir(1, 0, 0)
    dir_y = gp_Dir(0, 1, 0)
    dir_z = gp_Dir(0, 0, 1)
    ax3 = gp_Ax3(center, dir_z, dir_x)

    # Compute actual section area
    csa = compute_section_area(shape)

    return {
        "frame": ax3,
        "center": center,
        "dir_x": dir_x,
        "dir_y": dir_y,
        "dir_z": dir_z,
        "half_x": hx,
        "half_y": hy,
        "half_z": hz,
        "extents": [2*hx, 2*hy, 2*hz],
        "section_area": csa
    }


In [11]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Core.gp import gp_Pnt

def make_origin_marker(point=gp_Pnt(0, 0, 0), radius=5.0):
    """
    Creates a sphere centered at the global origin for visualization.
    """
    return BRepPrimAPI_MakeSphere(point, radius).Shape()


In [12]:
from OCC.Core.gp import gp_Pnt, gp_Vec
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def make_arrow(start: gp_Pnt, direction: gp_Dir, length=100.0):
    """
    Creates a line-shaped arrow from a start point in the given direction.
    """
    vec = gp_Vec(direction) * length
    end = gp_Pnt(start.XYZ())
    end.Translate(vec)
    return BRepBuilderAPI_MakeEdge(start, end).Shape()


In [13]:
from OCC.Core.gp import gp_Ax3, gp_Dir, gp_Pnt, gp_Trsf, gp_Vec
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def align_obb_to_dstv_frame(shape, origin_local, dir_x, dir_y, dir_z):
    """
    Align a solid into the DSTV frame:
    1. Translates the origin to (0, 0, 0)
    2. Rotates the axes to X = length, Y = height, Z = width (DSTV frame)

    Parameters:
    - shape: the OCC shape to transform
    - origin_local: gp_Pnt, the local rear-bottom-left corner
    - dir_x, dir_y, dir_z: gp_Dir, aligned local axes (length, height, width)

    Returns:
    - Transformed shape aligned with DSTV frame
    - Final rotation transform
    """

    # STEP 1: Translate shape so origin_local becomes (0, 0, 0)
    translate_vec = gp_Vec(origin_local, gp_Pnt(0, 0, 0))
    trsf_translate = gp_Trsf()
    trsf_translate.SetTranslation(translate_vec)

    shape_translated = BRepBuilderAPI_Transform(shape, trsf_translate, True).Shape()

    # STEP 2: Rotate to align local frame with DSTV global frame
    # Local frame with current axes
    local_frame = gp_Ax3(gp_Pnt(0, 0, 0), dir_z, dir_x)  # Z = up, X = length
    # Target DSTV frame
    dstv_frame = gp_Ax3(gp_Pnt(0, 0, 0), gp_Dir(0, 0, 1), gp_Dir(1, 0, 0))  # Z up, X forward

    trsf_rotate = gp_Trsf()
    trsf_rotate.SetDisplacement(local_frame, dstv_frame)

    shape_rotated = BRepBuilderAPI_Transform(shape_translated, trsf_rotate, True).Shape()

    return shape_rotated, trsf_rotate


In [14]:
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.gp import gp_Pnt, gp_Dir

def compute_obb_geometry(aligned_shape):
    """
    Compute extents and axes of a shape aligned in its own OBB frame.
    Returns length, height, width and the direction vectors.
    """
    # Step 1: Get the bounding box in the aligned frame
    aabb = Bnd_Box()
    brepbndlib.Add(aligned_shape, aabb)
    xmin, ymin, zmin, xmax, ymax, zmax = aabb.Get()

    # Step 2: Compute dimensions
    length = xmax - xmin
    height = ymax - ymin
    width = zmax - zmin

    # Step 3: Use world axes since shape is aligned
    dir_x = gp_Dir(1, 0, 0)
    dir_y = gp_Dir(0, 1, 0)
    dir_z = gp_Dir(0, 0, 1)

    # Step 4: Compute center
    center = gp_Pnt(
        (xmin + xmax) / 2,
        (ymin + ymax) / 2,
        (zmin + zmax) / 2
    )

    return {
        "aligned_extents": [length, height, width],
        "aligned_dir_x": dir_x,
        "aligned_dir_y": dir_y,
        "aligned_dir_z": dir_z,
        "aligned_center": center
    }


In [15]:
import numpy as np

def ensure_right_handed(dir_x, dir_y, dir_z):
    x = np.array([dir_x.X(), dir_x.Y(), dir_x.Z()])
    y = np.array([dir_y.X(), dir_y.Y(), dir_y.Z()])
    z = np.array([dir_z.X(), dir_z.Y(), dir_z.Z()])

    if np.dot(np.cross(x, y), z) < 0:
        print("⚠️ Detected left-handed system. Reversing Z to enforce right-handed convention.")
        dir_z = gp_Dir(-dir_z.X(), -dir_z.Y(), -dir_z.Z())

    return dir_x, dir_y, dir_z


In [16]:
def compute_section_area(solid):
    props = GProp_GProps()
    brepgprop.VolumeProperties(solid, props)
    volume = props.Mass()

    bbox = Bnd_Box()
    brepbndlib.Add(solid, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
    length = xmax - xmin  # assuming length is X after alignment

    area = volume / length if length > 0 else 0
    return area


In [17]:
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox
from OCC.Core.gp import gp_Pnt

def make_reference_box_at_origin(extents, color="YELLOW"):
    """
    Create a wireframe or solid reference box at origin with given extents.
    extents: [length, height, width] → corresponds to [X, Y, Z]
    """
    length, height, width = extents
    box = BRepPrimAPI_MakeBox(length, height, width).Shape()
    return box


In [18]:
if __name__ == "__main__":

    step_path = "../data/0444-1 ANGLED.step"
    # step_path = "../data/ncTest.step"
    # step_path = "../data/TestEA.step"
    # step_path = "../data/TestEAMirror.step"
    # step_path = "../data/TestUEA.step"
    # step_path = "../data/TestUEAMirror.step"
    # step_path = "../data/TestPFC.step"
    shape_orig = load_step(step_path)

    # visualize(shape_orig, viewer, color="GRAY")

    # STEP 1: Align to major geometry (robust PCA/face-based)
    primary_aligned_shape, trsf, cs, largest_face, dir_x, dir_y, dir_z = robust_align_solid_from_geometry(shape_orig)
    # visualize(primary_aligned_shape, viewer, color="PSS_LIGHT_BLUE")
    dir_x, dir_y, dir_z = ensure_right_handed(dir_x, dir_y, dir_z)

    # STEP 2: Compute OBB geometry
    obb_geom = compute_obb_geometry(primary_aligned_shape)
    aligned_extents = obb_geom["aligned_extents"]  # [length, height, width]
    aligned_center = obb_geom["aligned_center"]
    aligned_dir_x = obb_geom["aligned_dir_x"]
    aligned_dir_y = obb_geom["aligned_dir_y"]
    aligned_dir_z = obb_geom["aligned_dir_z"]

    # STEP 3: Compute section area
    section_area = compute_section_area(primary_aligned_shape)

    # STEP 4: Build classification signature
    cs = {
        "span_web": aligned_extents[1],
        "span_flange": aligned_extents[2],
        "area": section_area,
        "length": aligned_extents[0]
    }

    # print("Classification input:")
    # print("  span_web   =", cs["span_web"])
    # print("  span_flange=", cs["span_flange"])
    # print("  area       =", cs["area"])
    # print("  length     =", cs["length"])

    # STEP 5: Match against JSON library
    profile_match = classify_profile(cs, json_path, tol_dim=1.0, tol_area=5)
    if profile_match is None:
        raise ValueError("❌ No matching profile found")

    # STEP 6: Handle swapped dimensions
    swap_axes = profile_match.get("Requires_rotation", False)
    if swap_axes:
        print("🔁 Swapping height/width axes to match profile classification")
        aligned_dir_y, aligned_dir_z = aligned_dir_z, aligned_dir_y
        aligned_extents[1], aligned_extents[2] = aligned_extents[2], aligned_extents[1]
        
        # ✅ Ensure handedness is maintained after swap
        aligned_dir_x, aligned_dir_y, aligned_dir_z = ensure_right_handed(aligned_dir_x, aligned_dir_y, aligned_dir_z)

    # STEP 7: Compute DSTV origin
    origin_local = compute_dstv_origin(aligned_center, aligned_extents, aligned_dir_x, aligned_dir_y, aligned_dir_z)

    # STEP 8: Align OBB to global DSTV frame using frame transform
    # Construct gp_Ax3 from origin and axis directions
    obb_frame = gp_Ax3(
        origin_local,
        aligned_dir_z,  # Y axis becomes "main" direction (Z is inferred from X × Y)
        aligned_dir_x   # X axis as the reference direction
    )
    
    aligned_shape, trsf = align_obb_to_dstv_frame(
        primary_aligned_shape,
        origin_local,
        aligned_dir_x,
        aligned_dir_y,
        aligned_dir_z
    )

    obb_final = compute_obb_geometry(aligned_shape)
    final_dir_x = obb_final["aligned_dir_x"]
    final_dir_y = obb_final["aligned_dir_y"]
    final_dir_z = obb_final["aligned_dir_z"]

    # STEP 8.5 - rcreate final location frame for use in hole positioning
    dstv_frame = gp_Ax3(
        gp_Pnt(0, 0, 0),
        gp_Dir(final_dir_z.XYZ()),  # Z as main (Y axis of DSTV)
        gp_Dir(final_dir_x.XYZ())   # X as length axis
    )
    
    # STEP 9: Visual debugging — origin and axes
    origin_marker = make_origin_marker(gp_Pnt(0, 0, 0))
    arrow_x = make_arrow(gp_Pnt(0, 0, 0), final_dir_x)  # X = length
    arrow_y = make_arrow(gp_Pnt(0, 0, 0), final_dir_y)  # Y = height
    arrow_z = make_arrow(gp_Pnt(0, 0, 0), final_dir_z)  # Z = width

    visualize(origin_marker, viewer, color="RED")
    visualize(arrow_x, viewer, color="RED")
    visualize(arrow_y, viewer, color="GREEN")
    visualize(arrow_z, viewer, color="BLUE")
    visualize(aligned_shape, viewer, color="PSS_DARK_BLUE")

    print_result_table(profile_match)


,Field,STEP File,JSON Spec
0,Designation,203x203x46,
1,Category,UC,
2,Profile Type,I,
3,Match Score,0.23,
4,Rotation Required,False,
5,Height (mm),203.20,203.20
6,Width (mm),203.60,203.60
7,CSA (mm²),5856.53,5870.00
8,Length (mm),2280.00,0.00
9,Mass (Kg),105.11,


In [19]:
# DSTV Plane projection

In [20]:
from OCC.Core.gp import gp_Pln, gp_Pnt, gp_Dir, gp_XYZ
from OCC.Display.SimpleGui import init_display
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeFace
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox

frame = dstv_frame
origin = frame.Location()
dir_x = frame.XDirection()
dir_y = frame.YDirection()
dir_z = frame.Direction()
length, height, width = obb_geom["aligned_extents"]

# Helper: build a plane face at a given origin and normal

def make_offset_face(center: gp_Pnt, normal: gp_Dir, axis_x: gp_Dir, size_x: float, size_y: float, offset: float):
    """
    Create a face centered at 'center', normal to 'normal', offset outward by 'offset',
    using 'axis_x' as the local X axis, and size_x/size_y as the face dimensions.
    """
    from OCC.Core.gp import gp_Vec, gp_Ax3, gp_Pln
    from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeFace

    # Apply offset along normal direction
    offset_vec = gp_Vec(normal) * offset
    offset_center = gp_Pnt(center.XYZ().Added(offset_vec.XYZ()))

    # Build the face on a defined plane frame
    frame = gp_Ax3(offset_center, normal, axis_x)
    plane = gp_Pln(frame)
    return BRepBuilderAPI_MakeFace(plane, -size_x/2, size_x/2, -size_y/2, size_y/2).Face()


# Configuration
visual_offset = 10.0  # mm
half_height = height / 2.0
half_width = width / 2.0
plane_center = gp_Pnt(
    origin.XYZ()
    .Added(dir_x.XYZ().Multiplied(length / 2.0))
    .Added(dir_y.XYZ().Multiplied(height / 2.0))
    .Added(dir_z.XYZ().Multiplied(width / 2.0))
)

# === FACE U: top flange, ZX plane, normal = Y+

face_u = make_offset_face(
    center=plane_center,
    normal=dir_y,  # Y+
    axis_x=dir_x,
    size_x=length,
    size_y=width,
    offset=height/2 + visual_offset
)

# === FACE O: bottom flange, ZX plane, normal = Y−

face_o = make_offset_face(
    center=plane_center,
    normal=gp_Dir(-dir_y.X(), -dir_y.Y(), -dir_y.Z()),  # Y−
    axis_x=dir_x,
    size_x=length,
    size_y=width,
    offset=height/2 + visual_offset
)

# === FACE V: web, XY plane, normal = Z+

face_v = make_offset_face(
    center=plane_center,
    normal=dir_z,
    axis_x=dir_x,
    size_x=length,
    size_y=height,
    offset=width/2 + visual_offset
)



# Visualize
visualize(face_v, viewer, color="RED", alpha=0.5)
visualize(face_o, viewer, color="GREEN", alpha=0.5)
visualize(face_u, viewer, color="BLUE", alpha=0.5)

In [21]:
# Start of hole detection

In [22]:
def format_hole_table(df):
    fmt_cols = {
        "Diameter (mm)": "{:.1f}",
        "X (mm)": "{:.1f}",
        "Y (mm)": "{:.1f}"
    }
    valid_cols = {col: fmt for col, fmt in fmt_cols.items() if col in df.columns}

    return (
        df.style
        .set_caption("Detected Holes by Face")
        .set_table_styles([{
            'selector': 'caption',
            'props': [('caption-side', 'top'), ('font-size', '16px'), ('font-weight', 'bold')]
        }])
        .format(valid_cols)
        .background_gradient(cmap="Blues", subset=list(valid_cols.keys()))
    )



In [23]:
def show_projected_markers_on_dstv_planes(df_holes, origin_dstv, dstv_frame, display, radius=4.0, offset=10.0):
    from OCC.Core.gp import gp_Vec, gp_Pnt

    # DSTV axis vectors
    X = gp_Vec(dstv_frame.XDirection())
    Y = gp_Vec(dstv_frame.YDirection())
    Z = gp_Vec(dstv_frame.Direction())

    color_map = {
        "U": "GREEN",
        "O": "BLUE",
        "V": "RED"
    }

    for _, row in df_holes.iterrows():
        code = row["Code"]
        x = row["X (mm)"]
        y = row["Y (mm)"]

        if code == "V":
            vec = X.Scaled(x) + Y.Scaled(y) + Z.Scaled(offset)
        elif code == "U":
            vec = X.Scaled(x) + Z.Scaled(y) + Y.Scaled(-offset)
        elif code == "O":
            vec = X.Scaled(x) + Z.Scaled(y) + Y.Scaled(offset)
        else:
            continue

        world_vec = gp_Vec(origin_dstv.XYZ()) + vec
        point = gp_Pnt(world_vec.X(), world_vec.Y(), world_vec.Z())
        marker = make_origin_marker(point=point, radius=radius)
        visualize(marker, display, color=color_map.get(code, "GRAY"), alpha=1.0, clear=False)


In [24]:
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GeomAbs import GeomAbs_Cylinder
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.TopoDS import topods
from OCC.Core.gp import gp_Vec
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib

import numpy as np
import pandas as pd

def classify_and_project_holes_dstv(solid, dstv_frame, origin_dstv):
    props = GProp_GProps()
    brepgprop.VolumeProperties(solid, props)
    cm = props.CentreOfMass()
    centroid = np.array([cm.X(), cm.Y(), cm.Z()])

    im = props.MatrixOfInertia()
    T = np.array([[im.Value(i, j) for j in (1, 2, 3)] for i in (1, 2, 3)])
    ev, evec = np.linalg.eigh(T)
    order = np.argsort(ev)
    dirs = [gp_Vec(*evec[:, i]) for i in order]
    L, F, W = [np.array([d.X(), d.Y(), d.Z()]) for d in dirs]

    bbox = Bnd_Box()
    brepbndlib.Add(solid, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
    spans = np.array([xmax - xmin, ymax - ymin, zmax - zmin])
    half_extents = 0.5 * np.sort(spans)[::-1]
    length, width, thickness = half_extents * 2

    origin_nc1 = (centroid
                  - half_extents[0] * L
                  - half_extents[1] * F
                  + half_extents[2] * W)

    tol_long = 0.7
    tol_web = 0.7
    hole_data = []
    explorer = TopExp_Explorer(solid, TopAbs_FACE)

    while explorer.More():
        face = topods.Face(explorer.Current())
        adaptor = BRepAdaptor_Surface(face, True)
        if adaptor.GetType() == GeomAbs_Cylinder:
            cyl = adaptor.Cylinder()
            axis_dir = np.array([cyl.Axis().Direction().X(),
                                 cyl.Axis().Direction().Y(),
                                 cyl.Axis().Direction().Z()])
            axis_dir /= np.linalg.norm(axis_dir)

            if abs(np.dot(axis_dir, L)) > tol_long:
                explorer.Next()
                continue

            pf = GProp_GProps()
            brepgprop.SurfaceProperties(face, pf)
            fc = np.array([pf.CentreOfMass().X(),
                           pf.CentreOfMass().Y(),
                           pf.CentreOfMass().Z()])

            if abs(np.dot(axis_dir, W)) > tol_web:
                code, rgb = "V", (1.0, 0.0, 0.0)
            else:
                side = np.dot(fc - centroid, F)
                code, rgb = ("O", (0.0, 1.0, 0.0)) if side > 0 else ("U", (0.0, 0.0, 1.0))

            hole_data.append((face, code, rgb, fc))
        explorer.Next()

    rows = []
    for idx, (face, code, rgb, fc) in enumerate(hole_data, start=1):
        adaptor = BRepAdaptor_Surface(face, True)
        diam = 2.0 * adaptor.Cylinder().Radius()
        v = fc - origin_nc1

        if code == "V":
            x = float(np.dot(v, L))
            y = float(np.dot(v, F))
        elif code == "U":
            x = float(np.dot(v, L))
            y = -float(np.dot(v, W))
        else:
            x = float(np.dot(v, L))
            y_raw = float(np.dot(v, W))
            y = 2 * half_extents[2] - y_raw

        rows.append({
            "Hole #": idx,
            "Code": code,
            "Diameter (mm)": round(diam, 2),
            "X (mm)": round(x, 2),
            "Y (mm)": round(y, 2)
        })

    df_holes = pd.DataFrame(rows)
    return df_holes, hole_data, origin_nc1, L, F, W


In [25]:
raw_df_holes, hole_data, origin_nc1, L, F, W = classify_and_project_holes_dstv(aligned_shape, dstv_frame, origin)
show_projected_markers_on_dstv_planes(raw_df_holes, origin, dstv_frame, viewer)
df_holes = format_hole_table(raw_df_holes)
display(df_holes)


,Hole #,Code,Diameter (mm),X (mm),Y (mm)
0,1,V,12.0,1999.5,51.8
1,2,V,12.0,1499.5,51.8
2,3,V,12.0,899.5,51.8
3,4,V,12.0,299.6,51.8
4,5,V,12.0,299.6,151.8
5,6,V,12.0,899.5,151.8
6,7,V,12.0,1499.5,151.8
7,8,V,12.0,1999.5,151.8
8,9,O,12.0,424.6,253.2
9,10,U,12.0,424.6,49.9


In [26]:

step_vals = profile_match["STEP"]

header_dict = {
    "Designation":       profile_match["Designation"],
    "Mass":              step_vals["mass"],
    "Height":            step_vals["height"],
    "Width":             step_vals["width"],
    "CSA":               step_vals["area"],
    "web_thickness":     step_vals["web_thickness"],
    "flange_thickness":  step_vals["flange_thickness"],
    "root_radius":       step_vals["root_radius"],
    "toe_radius":        step_vals["toe_radius"],
    "code_profile":      profile_match["Profile_type"],
    "Length":            length
}

# %%
"""
Jupyter Notebook: Write DSTV NC1 ST-block header to .nc1 file with specified order
"""
# Cell 1: Define DSTV header fields inline
# Replace placeholder values with your actual data
project_number     = 'PROJ-123'
out_filename        = '0444-1 20250606'
model_filename     = 'model.step'
material_grade     = 'S355JR'
quantity           = 1

faces = ['V', 'U', 'O']  #Face priority order
path = '..\\data\\output\\'
# %%

filename = f"{path}{out_filename}.nc1"
with open(filename, 'w') as f:
    f.write('ST\n')
    f.write(f"  Project Number\n")
    f.write(f"  Drawing Number\n")
    f.write(f"  {out_filename}\n")
    f.write(f"  {model_filename}\n")
    f.write(f"  {material_grade}\n")
    f.write(f"  {quantity}\n")
    f.write(f"  {header_dict['Designation']}\n")
    f.write(f"  {header_dict['code_profile']}\n")
    f.write(f"    {header_dict['Length']:8.2f}\n")
    f.write(f"    {header_dict['Height']:8.2f}\n")
    f.write(f"    {header_dict['Width']:8.2f}\n")
    f.write(f"    {header_dict['flange_thickness']:8.2f}\n")
    f.write(f"    {header_dict['web_thickness']:8.2f}\n")
    f.write(f"    {header_dict['root_radius']:8.2f}\n")
    f.write(f"    {header_dict['Mass']:8.2f}\n")
    f.write('        0.00\n') #surface area
    # Following the spec, three zeros
    f.write('        0.00\n')
    f.write('        0.00\n')
    f.write('        0.00\n')
    f.write('        0.00\n')
    f.write('  -\n')
    f.write('  -\n')
    f.write('  -\n')
    f.write('  -\n')
    
    # BO blocks by face
    for face in faces:
        df_face = raw_df_holes[raw_df_holes['Code'] == face]
        if df_face.empty:
            continue
        f.write('BO\n')
        for _, row in df_face.iterrows():
            x = row['X (mm)']
            y = row['Y (mm)']
            d = row['Diameter (mm)']
            # Right-align numeric columns: x, y (8-wide), diameter (6-wide)
            f.write(f"  {face.lower()}  {x:8.2f} {y:8.2f} {d:6.2f}\n")
            
    f.write('EN\n')
print(f"DSTV header written to {filename}")

DSTV header written to ..\data\output\0444-1 20250606.nc1
